# Use of a Neural-Network for Constitutive Law

Import all the useful libraries before first run
We need here the classic ones such as:
- math
- numpy
- pandas
- matplotlib

And for the Neural Network, we also need to import parts of the keras module of TensorFlow

In [ ]:
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import exp,log

## Load the test database

Read the Test database

In [ ]:
dataPath = 'ANN-JohnsonCook'
read = np.load(dataPath + '/DatatestWithDerivatives.npz')
testData = read['testData']
nrows = testData.shape[0]
eps_test = testData[:,0].reshape(nrows,1)
epsp_test = testData[:,1].reshape(nrows,1)
T_test = testData[:,2].reshape(nrows,1)
sig_test = testData[:,3].reshape(nrows,1)

 ## Load the NN parameters

In [ ]:
ANN_name = '3-15-7-1-sigmoid'
#ANN_name = '3-7-4-1-sigmoid'

NN = np.load(dataPath + '/' + ANN_name + '.npz')
print (NN.files)
w1 = NN['w1']
w2 = NN['w2']
w3 = NN['w3']
b1 = NN['b1']
b2 = NN['b2']
b3 = NN['b3']
minEntries = NN['minEntries']
maxEntries = NN['maxEntries']
rangeEntries = maxEntries - minEntries
logBase = NN['logBase']
w1, b1, w2, b2, w3, b3, minEntries, maxEntries, rangeEntries, logBase

## Reshape data

In [ ]:
epsp_test = np.log(epsp_test / logBase)
inputTest = (np.hstack([eps_test, epsp_test, T_test]) - np.array(minEntries)[0:3]) / np.array(rangeEntries)[0:3]
inputTest

## Functions to compute the ANN

In [ ]:
def tanh2Layers(inputData):
    l1 = w1.dot(inputData) + b1
    f1 = np.tanh(l1)
    l2 = w2.dot(f1) + b2
    f2 = np.tanh(l2)
    sigP = w3.dot(f2) + b3
    SigmaNN = sigP * rangeEntries[3] + minEntries[3]
    return SigmaNN

def tanhPrime2Layers(x):
    w3v = w3.reshape(w3.shape[1],1)
    tanhx = np.tanh(w1.dot(x) + b1)
    p2 = w3v * (1 - np.tanh(w2.dot(tanhx) + b2)**2)
    p3 = (w2.T).dot(p2)
    p5 = p3 * (1 - tanhx**2)
    s = (w1.T).dot(p5)
    return s

In [ ]:
def sigmoid2Layers(inputData):
    l1 = w1.dot(inputData) + b1
    f1 = 1/(1 + np.exp(-l1))
    l2 = w2.dot(f1) + b2
    f2 = 1/(1 + np.exp(-l2))
    sigP = w3.dot(f2) + b3
    SigmaNN = sigP * rangeEntries[3] + minEntries[3]
    return SigmaNN

def sigmoidPrime2Layers(x):
    w3v = w3.reshape(w3.shape[1],1)
    expx = np.exp(-(w1.dot(x) + b1))
    exp2 = np.exp(w2.dot(1/(1 + expx)) + b2)
    p1 = w3v*(exp2 / (1 + exp2)**2)
    p2 = expx / (1 + expx)**2
    s = (w1.T).dot((w2.T).dot(p1) * p2)
    return s

Setup data

In [ ]:
inp = inputTest.T
sig = sig_test
#inp, sig

Rebuild the real $\sigma$

In [ ]:
if ('tanh' in ANN_name) : SigmaNN = tanh2Layers(inp)
elif ('sigmoid' in ANN_name) : SigmaNN = sigmoid2Layers(inp)
else : SigmaNN = 0
SigmaNN

In [ ]:
if ('tanh' in ANN_name) : SigmaNNP = tanhPrime2Layers(inp)
elif ('sigmoid' in ANN_name) : SigmaNNP = sigmoidPrime2Layers(inp)
else : SigmaNNP = 0
SigmaNNP

## Generation of Fortran code

In [ ]:
var = "%.12f"
comment = 'C '
cline = 'C **********************************************************************\n'
block =   '      '

def splitFortranLine(line):
    line=line.replace('| +',' +\n     + ')
    line=line.replace('| -',' -\n     + ')
    return line

def toPython(line):
    if (line[0]=='C'): line='#'+line[1:]
    line = line.replace('|','')
    line = line.replace("--", "+")
    line = line.replace("+-", "-")
    line = line.replace("(+", "(")
    line = line.replace('+',' + ')
    line = line.replace('-',' - ')
    line = line.replace('=  - ','=-')
    line = line.replace('( - ','(-')
    line = line.replace('  +  ',' + ')
    line = line.replace('.000000000000','.0')
    line = line + '\n'
    return line

def toFortran(line):
    line = line.replace("--", "+")
    line = line.replace("+-", "-")
    line = line.replace("(+", "(")
    line = line.replace('+',' + ')
    line = line.replace('-',' - ')
    line = line.replace('=  - ','=-')
    line = line.replace('( - ','(-')
    line = line.replace('  +  ',' + ')
    line = line.replace('.000000000000','.0')
    if (line[0]=='C'): line = line + '\n'
    else: line = block + line + '\n'
    line = splitFortranLine(line)
    return line

def vuhardHead():
    code = cline
    code += comment + 'Function to compute the ANN : ' + ANN_name + ' yield stress\n'
    code += cline
    code += '      subroutine vuhard (\n'
    code += 'C Read only -\n'
    code += '     +  nblock, nElement, nIntPt, nLayer, nSecPt, lAnneal, stepTime,\n'
    code += '     +  totalTime, dt, cmname, nstatev, nfieldv, nprops, props,\n'
    code += '     +  tempOld, tempNew, fieldOld, fieldNew, stateOld, eqps, eqpsRate,\n'
#     code += '     1     nblock,\n'
#     code += '     2     nElement, nIntPt, nLayer, nSecPt,\n'
#     code += '     3     lAnneal, stepTime, totalTime, dt, cmname,\n'
#     code += '     4     nstatev, nfieldv, nprops,\n'
#     code += '     5     props, tempOld, tempNew, fieldOld, fieldNew,\n'
#     code += '     6     stateOld,\n'
#     code += '     7     eqps, eqpsRate,\n'
    code += 'C Write only -\n'
    code += '     +  yield, dyieldDtemp, dyieldDeqps, stateNew)\n'
#     code += '     8     yield, dyieldDtemp, dyieldDeqps,\n'
#     code += '     9     stateNew )\n'
    code += 'C\n'
    code += "      include 'vaba_param.inc'\n"
    code += 'C\n'
#     code += '      dimension nElement(nblock),\n'
    code += '      dimension nElement(nblock), props(nprops), tempOld(nblock),\n'
    code += '     +  fieldOld(nblock,nfieldv), stateOld(nblock,nstatev),\n'
    code += '     +  tempNew(nblock), fieldNew(nblock,nfieldv), eqps(nblock),\n'
    code += '     +  eqpsRate(nblock), yield(nblock), dyieldDtemp(nblock),\n'
    code += '     +  dyieldDeqps(nblock,2), stateNew(nblock,nstatev)\n'
#     code += '     1     props(nprops),\n'
#     code += '     2     tempOld(nblock),\n'
#     code += '     3     fieldOld(nblock,nfieldv),\n'
#     code += '     4     stateOld(nblock,nstatev),\n'
#     code += '     5     tempNew(nblock),\n'
#     code += '     6     fieldNew(nblock,nfieldv),\n'
#     code += '     7     eqps(nblock),\n'
#     code += '     8     eqpsRate(nblock),\n'
#     code += '     9     yield(nblock),\n'
#     code += '     1     dyieldDtemp(nblock),\n'
#     code += '     2     dyieldDeqps(nblock,2),\n'
#     code += '     3     stateNew(nblock,nstatev)\n'
    code += 'C\n'
    code += '      character*80 cmname\n'
    code += 'C\n'
    code += '      do k = 1, nblock\n'
    return code

def vuhardTail():
    code = '      end do\n'
    code += 'C\n'
    code += '      return\n'
    code += '      end\n'
    return code

def encodeFunction(f,p,func):
    f += toFortran(func)
    p += toPython(func)
    return f, p

In [ ]:
code = ''
test = ''

code += vuhardHead()

code, test = encodeFunction(code, test, comment + 'xepsp = (eqps(k)-minEntries[0]) / rangeEntries[0]')
code, test = encodeFunction(code, test, 'xepsp = (eqps(k)-' + var%(minEntries[0]) + ')/' + var%(rangeEntries[0]))

code, test = encodeFunction(code, test, comment + 'xdepsp = (log(depsp/logBase)-minEntries[1]) / rangeEntries[1]')
code, test = encodeFunction(code, test, 'xdepsp = (log(eqpsRate(k)/' + var%(logBase[0]) + ')-' + var%(minEntries[1]) + ')/' + var%(rangeEntries[1]))

code, test = encodeFunction(code, test, comment + 'xtemp = (temp-minEntries[2]) / rangeEntries[2]')
code, test = encodeFunction(code, test, 'xtemp = (tempNew(k)-' + var%(minEntries[2])+')/' + var%(rangeEntries[2]))

code, test = encodeFunction(code, test, comment + 'za = np.exp(-w1.dot(x) - b1)')
for i in range(b1.shape[0]):
    code, test = encodeFunction(code, test, 'za' + str(i) + " = exp(-" + var%(w1[i,0]) + '*xepsp-' + var%(w1[i,1]) + '*xdepsp|-' + var%(w1[i,2]) + '*xtemp-' + var%(b1[i,0]) + ')')

code, test = encodeFunction(code, test, comment + 'zb = 1 + za')
for i in range(b1.shape[0]):
    code, test = encodeFunction(code, test, 'zb' + str(i) + " = 1.0 + za"+str(i))

code, test = encodeFunction(code, test, comment + 'zc = w2.dot(zb) + b2')
for i in range(b2.shape[0]):
    line = 'zc' + str(i) + ' = exp(-'
    for j in range(w2.shape[1]):
        if (j!=0): line += '|-'
        line += var%(w2[i,j]) + '/zb' + str(j)
    line += "|-" + var%(b2[i,0]) + ')'
    code, test = encodeFunction(code, test, line)
    
# code, test = encodeFunction(code, test, comment + 'xd = exp(-zc)')
# for i in range(b2.shape[0]):
#     line = 'xd'+str(i) + ' = exp(-zc' + str(i) + ')'
#     code, test = encodeFunction(code, test, line)
    
code, test = encodeFunction(code, test, comment + 'y = w3.dot(xd) + b3')
line = 'y = '
for j in range(w3.shape[1]):
    if (j!=0): line += '|+'
    line += var%(w3[0,j]) + '/(1.0 + zc'+str(j)+')'
line+="|+" + var%(b3[0,0])
code, test = encodeFunction(code, test, line)

code, test = encodeFunction(code, test, comment + 'zd = w3v*(xd / (1 + zc)**2)')
for i in range(b2.shape[0]):
    line = 'zd' + str(i) + ' = ' + var%(w3[0,i]) + '*(zc'+str(i) + '/(1.0 + zc' + str(i) + ')**2)'
    code, test = encodeFunction(code, test, line)

code, test = encodeFunction(code, test, comment + 'ze = za / (1 + za)**2')
for i in range(b1.shape[0]):
    line = 'ze' + str(i) + ' = za'+str(i) + ' / zb'+str(i) + '**2'
    code, test = encodeFunction(code, test, line)

code, test = encodeFunction(code, test, comment + 'zf = (w2.T).dot(zd) * xd')
for i in range(b1.shape[0]):
    line = 'zf' + str(i) + ' = ('
    for j in range(b2.shape[0]):
        if(j>0): line += '|+'
        line += var%(w2[j,i]) + '*zd' + str(j)
    line += ')*ze' + str(i)
    code, test = encodeFunction(code, test, line)
    
code, test = encodeFunction(code, test, comment + 'yd = (w1.T).dot(zf)')
for i in range(w1.shape[1]):
    line = 'yd' + str(i) + ' = '
    for j in range(w1.shape[0]):
        if (j>0): line += '|+'
        line += var%(w1[j,i]) + '*zf' + str(j)
    code, test = encodeFunction(code, test, line)

line = 'Yield(k) = ' + var%(rangeEntries[3]) + '*y+' + var%(minEntries[3])
code, test = encodeFunction(code, test, line)

line = 'dyieldDeqps(k,1) = ' + var%(rangeEntries[3]/rangeEntries[0]) + '*yd0'
code, test = encodeFunction(code, test, line)
line = 'dyieldDeqps(k,2) = ' + var%(rangeEntries[3]/rangeEntries[1]) + '*yd1 / eqpsRate(k)'
code, test = encodeFunction(code, test, line)
line = 'dyieldDtemp(k) = ' + var%(rangeEntries[3]/rangeEntries[2]) + '*yd2'
code, test = encodeFunction(code, test, line)

code += vuhardTail()

print(test)

In [ ]:
dataTestLine = 1
eqps = testData[dataTestLine,0]
eqpsRate = testData[dataTestLine,1]
tempNew = testData[dataTestLine,2]

### Paste content of test here after

In [ ]:
# xepsp = (eqps(k) - minEntries[0]) / rangeEntries[0]
xepsp = (eqps - 0.0)/1.0
# xdepsp = (log(depsp/logBase) - minEntries[1]) / rangeEntries[1]
xdepsp = (log(eqpsRate/1.0) - 0.0)/10.819778284410
# xtemp = (temp - minEntries[2]) / rangeEntries[2]
xtemp = (tempNew - 20.0)/480.0
# xa = np.exp(-w1.dot(x)  -  b1)

# za = np.exp(-w1.dot(x)  -  b1)
za0 = exp(0.171193018556*xepsp + 0.498235881329*xdepsp - 1.572309255600*xtemp + 0.549710810184)
za1 = exp(0.182183891535*xepsp + 0.279642552137*xdepsp - 1.381547451019*xtemp + 1.007667899132)
za2 = exp(0.121008850634*xepsp + 0.093622177839*xdepsp + 1.284856081009*xtemp - 1.146428942680)
za3 = exp(-0.677952528000*xepsp - 0.785129725933*xdepsp + 3.600407361984*xtemp + 1.056580066681)
za4 = exp(8.224933624268*xepsp - 0.102857425809*xdepsp + 1.170001864433*xtemp + 0.834357798100)
za5 = exp(0.262212038040*xepsp + 0.370604902506*xdepsp - 1.530593752861*xtemp + 0.557993113995)
za6 = exp(-0.363313168287*xepsp - 0.307944297791*xdepsp + 1.739808201790*xtemp + 0.515264987946)
za7 = exp(0.171282619238*xepsp + 0.936020970345*xdepsp + 0.700896143913*xtemp + 0.525115251541)
za8 = exp(2.934058904648*xepsp - 0.024969343096*xdepsp + 0.818501532078*xtemp - 0.164255663753)
za9 = exp(0.298778355122*xepsp + 0.517931699753*xdepsp + 7.580852508545*xtemp + 1.864312648773)
za10 = exp(-1.177027940750*xepsp + 0.308994054794*xdepsp + 1.684179663658*xtemp + 0.859441757202)
za11 = exp(0.231339216232*xepsp + 0.290423154831*xdepsp - 1.502451419830*xtemp + 1.198348641396)
za12 = exp(-235.784103393555*xepsp + 0.099293000996*xdepsp - 0.760096788406*xtemp - 0.819592595100)
za13 = exp(23.344444274902*xepsp - 0.164729803801*xdepsp + 0.949119210243*xtemp + 0.500462353230)
za14 = exp(-0.329279005527*xepsp - 0.054267201573*xdepsp + 1.555212616920*xtemp + 0.580829739571)
# zb = 1 + za
zb0 = 1.0 + za0
zb1 = 1.0 + za1
zb2 = 1.0 + za2
zb3 = 1.0 + za3
zb4 = 1.0 + za4
zb5 = 1.0 + za5
zb6 = 1.0 + za6
zb7 = 1.0 + za7
zb8 = 1.0 + za8
zb9 = 1.0 + za9
zb10 = 1.0 + za10
zb11 = 1.0 + za11
zb12 = 1.0 + za12
zb13 = 1.0 + za13
zb14 = 1.0 + za14
# zc = w2.dot(zb) + b2
zc0 = exp(-0.000755758665/zb0 - 0.348879784346/zb1 + 1.622294902802/zb2 + 1.744126558304/zb3 - 1.189603447914/zb4 - 0.773437738419/zb5 + 1.077992796898/zb6 - 0.195966541767/zb7 - 0.371459305286/zb8 + 2.175866603851/zb9 - 0.267415910959/zb10 - 0.117423757911/zb11 + 1.837396621704/zb12 + 1.631284832954/zb13 + 0.915312469006/zb14 + 0.220450609922)
zc1 = exp(1.520736217499/zb0 + 0.598750412464/zb1 + 0.567928969860/zb2 - 0.733987450600/zb3 + 1.956816315651/zb4 + 0.990542054176/zb5 + 0.024108286947/zb6 + 0.921862065792/zb7 + 1.585801959038/zb8 + 0.359389543533/zb9 + 0.568457961082/zb10 + 0.462055325508/zb11 - 0.937371969223/zb12 - 0.713293492794/zb13 + 0.270313948393/zb14 + 0.580874204636)
zc2 = exp(0.263961493969/zb0 - 0.340663641691/zb1 + 1.086898207664/zb2 + 0.190290063620/zb3 - 1.526942133904/zb4 - 0.134074971080/zb5 + 0.458628684282/zb6 + 0.210141628981/zb7 - 0.690639853477/zb8 + 0.559289455414/zb9 + 0.561830639839/zb10 - 0.010643188842/zb11 + 2.779141187668/zb12 - 2.203900575638/zb13 + 0.874182760715/zb14 + 0.327307760715)
zc3 = exp(1.667145609856/zb0 + 1.261272549629/zb1 + 1.568691968918/zb2 + 1.300865530968/zb3 + 0.965722560883/zb4 + 1.076323390007/zb5 + 0.399949252605/zb6 + 2.976247072220/zb7 + 3.980113983154/zb8 + 1.212576389313/zb9 + 0.478132873774/zb10 + 1.447541594505/zb11 + 0.287288188934/zb12 + 0.847129225731/zb13 + 0.899507462978/zb14 + 0.965191483498)
zc4 = exp(0.421280384064/zb0 + 0.564329922199/zb1 - 1.009604096413/zb2 - 0.335408806801/zb3 + 0.515781700611/zb4 + 0.189889833331/zb5 - 0.561685442924/zb6 + 0.274590790272/zb7 + 0.640589475632/zb8 + 0.454831272364/zb9 - 0.185221791267/zb10 + 1.204036474228/zb11 - 2.715961456299/zb12 + 2.415366411209/zb13 - 0.508277475834/zb14 - 0.001537762932)
zc5 = exp(1.879916071892/zb0 + 1.522165775299/zb1 + 0.468759000301/zb2 - 1.417994260788/zb3 - 0.370010912418/zb4 + 2.718472957611/zb5 - 0.469961762428/zb6 + 2.314279079437/zb7 + 0.682480514050/zb8 + 1.616387128830/zb9 - 0.202725619078/zb10 + 1.451031565666/zb11 - 0.099444560707/zb12 + 1.293660640717/zb13 - 0.203722134233/zb14 + 0.991643249989)
zc6 = exp(0.589749693871/zb0 + 0.571380496025/zb1 - 0.120540380478/zb2 - 0.763658463955/zb3 + 1.608781814575/zb4 + 0.913990020752/zb5 - 0.719076871872/zb6 + 1.206779360771/zb7 + 0.747950553894/zb8 + 0.689776837826/zb9 + 0.001232341980/zb10 + 0.592846393585/zb11 - 1.381900072098/zb12 - 0.327141195536/zb13 - 0.125081762671/zb14 + 0.179510131478)
# y = w3.dot(xd) + b3
y =-1.147175073624/(1.0 + zc0) + 0.396453082561/(1.0 + zc1) - 1.731730222702/(1.0 + zc2) + 0.341490298510/(1.0 + zc3) + 0.917140424252/(1.0 + zc4) + 0.416720062494/(1.0 + zc5) + 0.469032645226/(1.0 + zc6) - 0.399738907814
# zd = w3v*(xd / (1 + zc)**2)
zd0 =-1.147175073624*(zc0/(1.0 + zc0)**2)
zd1 = 0.396453082561*(zc1/(1.0 + zc1)**2)
zd2 =-1.731730222702*(zc2/(1.0 + zc2)**2)
zd3 = 0.341490298510*(zc3/(1.0 + zc3)**2)
zd4 = 0.917140424252*(zc4/(1.0 + zc4)**2)
zd5 = 0.416720062494*(zc5/(1.0 + zc5)**2)
zd6 = 0.469032645226*(zc6/(1.0 + zc6)**2)
# ze = za / (1 + za)**2
ze0 = za0 / zb0**2
ze1 = za1 / zb1**2
ze2 = za2 / zb2**2
ze3 = za3 / zb3**2
ze4 = za4 / zb4**2
ze5 = za5 / zb5**2
ze6 = za6 / zb6**2
ze7 = za7 / zb7**2
ze8 = za8 / zb8**2
ze9 = za9 / zb9**2
ze10 = za10 / zb10**2
ze11 = za11 / zb11**2
ze12 = za12 / zb12**2
ze13 = za13 / zb13**2
ze14 = za14 / zb14**2
# zf = (w2.T).dot(zd) * xd
zf0 = (0.000755758665*zd0 - 1.520736217499*zd1 - 0.263961493969*zd2 - 1.667145609856*zd3 - 0.421280384064*zd4 - 1.879916071892*zd5 - 0.589749693871*zd6)*ze0
zf1 = (0.348879784346*zd0 - 0.598750412464*zd1 + 0.340663641691*zd2 - 1.261272549629*zd3 - 0.564329922199*zd4 - 1.522165775299*zd5 - 0.571380496025*zd6)*ze1
zf2 = (-1.622294902802*zd0 - 0.567928969860*zd1 - 1.086898207664*zd2 - 1.568691968918*zd3 + 1.009604096413*zd4 - 0.468759000301*zd5 + 0.120540380478*zd6)*ze2
zf3 = (-1.744126558304*zd0 + 0.733987450600*zd1 - 0.190290063620*zd2 - 1.300865530968*zd3 + 0.335408806801*zd4 + 1.417994260788*zd5 + 0.763658463955*zd6)*ze3
zf4 = (1.189603447914*zd0 - 1.956816315651*zd1 + 1.526942133904*zd2 - 0.965722560883*zd3 - 0.515781700611*zd4 + 0.370010912418*zd5 - 1.608781814575*zd6)*ze4
zf5 = (0.773437738419*zd0 - 0.990542054176*zd1 + 0.134074971080*zd2 - 1.076323390007*zd3 - 0.189889833331*zd4 - 2.718472957611*zd5 - 0.913990020752*zd6)*ze5
zf6 = (-1.077992796898*zd0 - 0.024108286947*zd1 - 0.458628684282*zd2 - 0.399949252605*zd3 + 0.561685442924*zd4 + 0.469961762428*zd5 + 0.719076871872*zd6)*ze6
zf7 = (0.195966541767*zd0 - 0.921862065792*zd1 - 0.210141628981*zd2 - 2.976247072220*zd3 - 0.274590790272*zd4 - 2.314279079437*zd5 - 1.206779360771*zd6)*ze7
zf8 = (0.371459305286*zd0 - 1.585801959038*zd1 + 0.690639853477*zd2 - 3.980113983154*zd3 - 0.640589475632*zd4 - 0.682480514050*zd5 - 0.747950553894*zd6)*ze8
zf9 = (-2.175866603851*zd0 - 0.359389543533*zd1 - 0.559289455414*zd2 - 1.212576389313*zd3 - 0.454831272364*zd4 - 1.616387128830*zd5 - 0.689776837826*zd6)*ze9
zf10 = (0.267415910959*zd0 - 0.568457961082*zd1 - 0.561830639839*zd2 - 0.478132873774*zd3 + 0.185221791267*zd4 + 0.202725619078*zd5 - 0.001232341980*zd6)*ze10
zf11 = (0.117423757911*zd0 - 0.462055325508*zd1 + 0.010643188842*zd2 - 1.447541594505*zd3 - 1.204036474228*zd4 - 1.451031565666*zd5 - 0.592846393585*zd6)*ze11
zf12 = (-1.837396621704*zd0 + 0.937371969223*zd1 - 2.779141187668*zd2 - 0.287288188934*zd3 + 2.715961456299*zd4 + 0.099444560707*zd5 + 1.381900072098*zd6)*ze12
zf13 = (-1.631284832954*zd0 + 0.713293492794*zd1 + 2.203900575638*zd2 - 0.847129225731*zd3 - 2.415366411209*zd4 - 1.293660640717*zd5 + 0.327141195536*zd6)*ze13
zf14 = (-0.915312469006*zd0 - 0.270313948393*zd1 - 0.874182760715*zd2 - 0.899507462978*zd3 + 0.508277475834*zd4 + 0.203722134233*zd5 + 0.125081762671*zd6)*ze14
# yd = (w1.T).dot(zf)
yd0 =-0.171193018556*zf0 - 0.182183891535*zf1 - 0.121008850634*zf2 + 0.677952528000*zf3 - 8.224933624268*zf4 - 0.262212038040*zf5 + 0.363313168287*zf6 - 0.171282619238*zf7 - 2.934058904648*zf8 - 0.298778355122*zf9 + 1.177027940750*zf10 - 0.231339216232*zf11 + 235.784103393555*zf12 - 23.344444274902*zf13 + 0.329279005527*zf14
yd1 =-0.498235881329*zf0 - 0.279642552137*zf1 - 0.093622177839*zf2 + 0.785129725933*zf3 + 0.102857425809*zf4 - 0.370604902506*zf5 + 0.307944297791*zf6 - 0.936020970345*zf7 + 0.024969343096*zf8 - 0.517931699753*zf9 - 0.308994054794*zf10 - 0.290423154831*zf11 - 0.099293000996*zf12 + 0.164729803801*zf13 + 0.054267201573*zf14
yd2 = 1.572309255600*zf0 + 1.381547451019*zf1 - 1.284856081009*zf2 - 3.600407361984*zf3 - 1.170001864433*zf4 + 1.530593752861*zf5 - 1.739808201790*zf6 - 0.700896143913*zf7 - 0.818501532078*zf8 - 7.580852508545*zf9 - 1.684179663658*zf10 + 1.502451419830*zf11 + 0.760096788406*zf12 - 0.949119210243*zf13 - 1.555212616920*zf14

Yield = 977.555715042962*y + 579.184642915415
dyieldDeqps1 = 977.555715042962*yd0
dyieldDeqps2 = 90.348959964501*yd1 / eqpsRate
dyieldDtemp = 2.036574406340*yd2

### Check that results are OK

In [ ]:
print('Yield        : %g %g' %(Yield, SigmaNN[0,dataTestLine]))
print('dyieldDeqps1 : %g %g' %(dyieldDeqps1, SigmaNNP[0,dataTestLine]*rangeEntries[3]/rangeEntries[0]))
print('dyieldDeqps2 : %g %g' %(dyieldDeqps2, SigmaNNP[1,dataTestLine]*rangeEntries[3]/rangeEntries[1]/testData[dataTestLine,1]))
print('dyieldDtemp  : %g %g' %(dyieldDtemp, SigmaNNP[2,dataTestLine]*rangeEntries[3]/rangeEntries[2]))

## Get the Fortran code

In [ ]:
print(code)

## Writes the fortran file

In [ ]:
text_file = open(dataPath+"/VUHARD-ANN-"+ANN_name+".f", "w")
text_file.write(code)
text_file.close()